In [9]:
import xlrd
import copy

import pandas as pd
import numpy as np

In [10]:
df1 = pd.read_csv('invitro.csv', sep=';')
df1.data_section_path[0]

'Биохимические исследования/Белки и аминокислоты'

In [10]:
df = pd.read_csv('analysis/moskva_analysis.csv')

In [15]:
df.data_prices[18]

'290 руб'

In [13]:
list(df.data_number).index('8')

18

In [16]:
290 * 1.6

464.0

### Helper

In [8]:
def get_notnullable_rows_by_col(data, col_name):
    return data[np.logical_not(data[col_name].isna())]

def get_analysis_sections(data, not_null_col, section_name_col, is_all_section_general=True):
    def is_general_section(name):
        """
            Предположение о том, что разделы пишутся с ЗАГЛАВНЫХ букв.
        """
        
        if name.upper() == name and not 'ВНИМАНИЕ' in name:
            return True
        
        return False
        
    sections_names = data[data[not_null_col].isna()][section_name_col]
    sections_index = list(sections_names.index) + [data.shape[0]]

    counts = [sections_index[i+1] - sections_index[i] - 1 for i in range(sections_names.shape[0])]
    
    if not is_all_section_general:
        last_general_section_ind = 0

        for ind, section_name in enumerate(sections_names):
            if not is_general_section(section_name):
                counts[last_general_section_ind] += counts[ind]
                counts[ind] = 0
            else:
                last_general_section_ind = ind

    analysis_sections = []

    for section_name, count in zip(sections_names, counts):
         analysis_sections.extend([section_name] * count)
            
    return analysis_sections

# Инвитро

In [5]:
invitro = pd.read_excel('ИНВИТРО.xls', header=None)
for ind in [0, 3, 4, 5, 8]:    
    del invitro[ind]

FileNotFoundError: [Errno 2] No such file or directory: 'ИНВИТРО.xls'

In [6]:
invitro.head()

NameError: name 'invitro' is not defined

In [5]:
invitro.to_csv('in.csv', index=False)

In [6]:
invitro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2063 entries, 0 to 2062
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1       2063 non-null   object 
 1   2       1871 non-null   object 
 2   6       1861 non-null   object 
 3   7       1871 non-null   float64
dtypes: float64(1), object(3)
memory usage: 64.6+ KB


In [7]:
len(get_analysis_sections(invitro, 7, 1))

1871

In [9]:
invitro_data = get_notnullable_rows_by_col(invitro, 7)
invitro_data.columns = ['code', 'name', 'period', 'price']
invitro_data['section'] = get_analysis_sections(invitro, 7, 1, is_all_section_general=False)

<ipython-input-9-3d912a4d3da2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invitro_data['section'] = get_analysis_sections(invitro, 7, 1, is_all_section_general=False)


In [10]:
invitro_sections = get_analysis_sections(invitro, 7, 1, is_all_section_general=False)
pd.Series(list(set(invitro_sections))).to_frame().to_excel('invitro_sections.xls', index=False)

In [11]:
for series_row in invitro_data.iloc:
    print(series_row.to_dict())
    break

{'code': '11HOMA', 'name': 'Индекс инсулинорезистентности (глюкоза, инсулин, расчетный индекс инсулинорезистентности HOMA- IR)', 'period': 'до 2', 'price': 540.0, 'section': 'ИМЕННЫЕ НАБОРЫ АНАЛИЗОВ ЛАБОРАТОРИЙ'}


In [12]:
invitro_data.to_csv('csv/invitro.csv', index=False)
invitro_data.to_excel('invitro.xls', index=False)

In [13]:
print("Всего анализов:", invitro_data.shape[0])

Всего анализов: 1871


In [9]:
df = pd.read_csv('csv/invitro.csv')

In [12]:
list(df.name).find('АЛаТ')

AttributeError: 'list' object has no attribute 'find'

# Ситилаб

In [14]:
citylab = pd.read_excel('ситилаб.xlsx')

In [15]:
citylab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1864 entries, 0 to 1863
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   КОД                  1863 non-null   object 
 1   НАИМЕНОВАНИЕ УСЛУГИ  1656 non-null   object 
 2   ТИП ИССЛ.            1656 non-null   object 
 3   БЛАНК                1656 non-null   object 
 4   БИОМАТ.              1656 non-null   object 
 5   СРОК*                1656 non-null   object 
 6   Цена 2020            1862 non-null   float64
dtypes: float64(1), object(6)
memory usage: 102.1+ KB


In [16]:
analysis_sections = get_analysis_sections(citylab, 'НАИМЕНОВАНИЕ УСЛУГИ', 'КОД')

In [17]:
citylab_sections = get_analysis_sections(citylab, 'НАИМЕНОВАНИЕ УСЛУГИ', 'КОД')
pd.Series(list(set(citylab_sections))).to_frame().to_excel('citylab_sections.xls', index=False)

In [18]:
citylab.head()

,КОД,НАИМЕНОВАНИЕ УСЛУГИ,ТИП ИССЛ.,БЛАНК,БИОМАТ.,СРОК*,Цена 2020
0,ГЕМАТОЛОГИЧЕСКИЕ ИССЛЕДОВАНИЯ,NaN,NaN,NaN,NaN,NaN,NaN
1,11-10-001,Общий анализ крови (CBC/Diff - 5 фракций лейко...,колич.,ОНБ,Кровь ЭДТА,1 р.д.,365.5
2,11-10-002,Ретикулоциты (проточная цитофлуорометрия),колич.,ОНБ,Кровь ЭДТА,1 р.д.,289.0
3,11-10-003,СОЭ (по Вестергрен),колич.,ОНБ,Кровь ЭДТА,1 р.д.,153.0
4,11-10-004,Лейкоцитарная формула (микроскопия),колич.,ОНБ,Кровь ЭДТА,1 р.д.,221.0


In [19]:
len(analysis_sections)

1656

In [20]:
citylab_data = get_notnullable_rows_by_col(citylab, 'СРОК*')
citylab_data['Раздел'] = get_analysis_sections(citylab, 'НАИМЕНОВАНИЕ УСЛУГИ', 'КОД', is_all_section_general=False)
citylab_data.columns = ['code', 'name', 'research_type', 'blank_form', 'sampling_type', 'period', 'price', 'section']

<ipython-input-20-52a7017f55d7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  citylab_data['Раздел'] = get_analysis_sections(citylab, 'НАИМЕНОВАНИЕ УСЛУГИ', 'КОД', is_all_section_general=False)


In [21]:
citylab_data.to_csv("csv/citylab.csv", index=False)

In [22]:
citylab_data.to_excel("citylab.xls", index=False)

In [23]:
print("Всего анализов:", citylab_data.shape[0])

Всего анализов: 1656


# Хеликс

In [24]:
helix = pd.read_excel("хеликс.xls")

In [25]:
helix.head()

,Артикул,Наименование,Цена (руб.),Срок выполнения
0,02. Общеклинические исследования,NaN,NaN,NaN
1,02-001,Анализ кала на скрытую кровь,110,2
2,02-002,Анализ мочи по Нечипоренко,175,1
3,02-003,Микроскопическое исследование отделяемого моче...,130,До 15:00 следующего дня
4,02-004,Микроскопическое исследование секрета предстат...,180,До 15:00 следующего дня


In [26]:
helix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1930 entries, 0 to 1929
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Артикул          1930 non-null   object
 1   Наименование     1911 non-null   object
 2   Цена (руб.)      1911 non-null   object
 3   Срок выполнения  1911 non-null   object
dtypes: object(4)
memory usage: 60.4+ KB


In [32]:
helix.head()

,Артикул,Наименование,Цена (руб.),Срок выполнения
0,02. Общеклинические исследования,NaN,NaN,NaN
1,02-001,Анализ кала на скрытую кровь,110,2
2,02-002,Анализ мочи по Нечипоренко,175,1
3,02-003,Микроскопическое исследование отделяемого моче...,130,До 15:00 следующего дня
4,02-004,Микроскопическое исследование секрета предстат...,180,До 15:00 следующего дня


In [27]:
helix_data = get_notnullable_rows_by_col(helix, 'Цена (руб.)')

In [28]:
helix_data['section'] = [section[4:] for section in get_analysis_sections(helix, 'Наименование', 'Артикул', is_all_section_general=True)]
helix_data.columns = ['code', 'name', 'price', 'period', 'section']

<ipython-input-28-14516fd62d49>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  helix_data['section'] = [section[4:] for section in get_analysis_sections(helix, 'Наименование', 'Артикул', is_all_section_general=True)]


In [29]:
helix_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1911 entries, 1 to 1929
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   code     1911 non-null   object
 1   name     1911 non-null   object
 2   price    1911 non-null   object
 3   period   1911 non-null   object
 4   section  1911 non-null   object
dtypes: object(5)
memory usage: 89.6+ KB


In [30]:
helix_sections = list(set([section[4:] for section in get_analysis_sections(helix, 'Наименование', 'Артикул', is_all_section_general=True)]))
pd.Series(list(set(helix_sections))).to_frame().to_excel('helix_sections.xls', index=False)

In [31]:
helix_data.to_csv('csv/helix.csv', index=False)

In [198]:
print("Всего анализов:", helix_data.shape[0])

Всего анализов: 1911


# Matched

In [15]:
matched = pd.read_excel('Analysis_connection_2(1).xlsx')

In [16]:
matched.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903 entries, 0 to 902
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Раздел                   902 non-null    object
 1   Название анализа         902 non-null    object
 2   Код Инвитро              897 non-null    object
 3   Цена Инвитро             897 non-null    object
 4   Ссылка Инвитро           897 non-null    object
 5   Код Хеликса              897 non-null    object
 6   Цена Хеликса             896 non-null    object
 7   Код Ситилаб              896 non-null    object
 8   Цена Ситилаб             896 non-null    object
 9   Ссылка Ситилаб           897 non-null    object
 10  Код Гемотеста            624 non-null    object
 11  Цена Гемотеста           624 non-null    object
 12  Ссылка Гемотест          624 non-null    object
 13  Код CMD                  624 non-null    object
 14  Цена CMD                 624 non-null    o

In [31]:
matched.to_csv('csv/matcheds.csv')

In [17]:
matched_dict = dict.fromkeys(["codes", "prepare", "sampling"])

for k, _ in matched_dict.items():
    matched_dict[k] = []

In [18]:
for row in matched.iloc:
    row = row.to_dict()
    matched_dict["codes"].append(";".join([str(row[name]) for name in ['Код Инвитро', 'Код Хеликса', 'Код Ситилаб',]]))
    matched_dict["prepare"].append(row['Подготовка к анализу'])
    matched_dict["sampling"].append(row['Тип взятия биоматериала'])

In [20]:
matched_df = pd.DataFrame(matched_dict)

In [21]:
matched_df.to_csv('csv/matched.csv', index=False)

In [22]:
print("Всего сравнений:", matched.shape[0])

Всего сравнений: 903


## Samplings

In [27]:
samplings_df = pd.read_excel('Заборы.xlsx')

In [28]:
samplings_df.head()

,Название на сайте,Название в БД,Стоимость
0,Взятие крови из периферической вены,Венозная кровь,220.0
1,Взятие крови из периферической вены,Сыв-ка крови,220.0
2,Взятие крови из периферической вены,Плазма с фторидом натрия,220.0
3,Взятие крови из периферической вены,"Сыв-ка крови, плазма с фторидом натрия",220.0
4,Взятие крови из периферической вены,ЭДТА,220.0


In [29]:
samplings_df.columns = ['site_name', 'name', 'price']

In [30]:
samplings_df.to_csv('csv/samplings.csv')

### Sections

In [55]:
df = pd.read_excel('sections_connection.xlsx')
df.columns = ['invitro_section', 'helix_section', 'citylab_section']
df.helix_section = [section[3:] if isinstance(section, str) else np.nan for section in df.helix_section ]
df = df.fillna('ДРУГОЕ')

In [56]:
df.to_csv('csv/sections_connections.csv', index=False)

In [ ]:
pd.read_csv('')

In [57]:
'Общеклинические исследования' == 'Общеклинические исследования'

True

In [4]:
import requests

In [2]:
headers = {'Content-Type': 'application/json', 'charset': 'UTF-8', 'Content-Length': '268', 'Chunked': 'false'}

data = {"TerminalKey":"1599305767668","OrderId":"87","Success":True,"Status":"CONFIRMED","PaymentId":342918623,"ErrorCode":"0","Amount":500,"CardId":49952352,"Pan":"427676******4888","ExpDate":"0822","Token":"599965d44e33329eb1c227c4427b2642071a867d02d64010c50c7611a258d7b4"}

In [5]:
r = requests.post('http://lotoslab.ru/payment/', 
    headers=headers,
    data=data)

In [6]:
r.content

b'OK'

In [29]:
df = pd.read_excel('Helix_adress_msk.xlsx')

In [33]:
df.columns = ['address', 'coord']

In [34]:
df.address = [n.replace('\n', ' ') for n in df.address]

In [35]:
df['latitude'] = [x.split(',')[0] for x in df.coord]
df['longitude'] = [x.split(',')[1] for x in df.coord]

In [36]:
df['color'] = ['red' for i in range(df.shape[0])]

In [41]:
df['lab_name'] = ['Лаборатория-партнёр' for i in range(df.shape[0])]
del df['coord']

In [42]:
df.to_csv('Locations/moskva/helix.csv', sep='\t', index=False)

In [23]:
df = pd.read_csv('analysis.csv')
df = df[:3]
df['price'][0] = 123
df['description'][0] = 'УЗДАЛЬ'
df.to_csv('csv/analysis.csv', index=False)

In [7]:
a = b'{"TerminalKey":"1599305767668","OrderId":"87","Success":true,"Status":"CONFIRMED","PaymentId":342918623,"ErrorCode":"0","Amount":500,"CardId":49952352,"Pan":"427676******4888","ExpDate":"0822","Token":"599965d44e33329eb1c227c4427b2642071a867d02d64010c50c7611a258d7b4"}'

In [10]:
eval(a.decode('utf-8').replace('true', 'True'))

{'TerminalKey': '1599305767668',
 'OrderId': '87',
 'Success': True,
 'Status': 'CONFIRMED',
 'PaymentId': 342918623,
 'ErrorCode': '0',
 'Amount': 500,
 'CardId': 49952352,
 'Pan': '427676******4888',
 'ExpDate': '0822',
 'Token': '599965d44e33329eb1c227c4427b2642071a867d02d64010c50c7611a258d7b4'}

In [17]:
def body_to_dict(a):
        body = a.decode('utf-8')
        replaces = {"true": "True", 'false': "False", 'none': "None"}
        for x in replaces:
            body = body.replace(x, replaces[x])

        return eval(body)

In [18]:
body_to_dict(a)

{'TerminalKey': '1599305767668',
 'OrderId': '87',
 'Success': True,
 'Status': 'CONFIRMED',
 'PaymentId': 342918623,
 'ErrorCode': '0',
 'Amount': 500,
 'CardId': 49952352,
 'Pan': '427676******4888',
 'ExpDate': '0822',
 'Token': '599965d44e33329eb1c227c4427b2642071a867d02d64010c50c7611a258d7b4'}

In [2]:
pd.read_csv('Locations/moskva/invitro.csv').to_csv('Locations/moskva/invitro.csv', sep='\t')